# Goals
- To prepare a mini version of the data sources for extracting the affliation of the papers. That is, the origin of the papers' data, which we assume the data is collected from the regions/countries that the authors are from.
- To identify the countries of the data origin for each paper. 

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import pandas as pd

# Get only the location information of each papers
- None of the columns are unique id, including cord_uid. But, we will use this for now because it has the most unique ids. 
- The "merge_raw_data.csv" is created by Panayiotis. Since it is very large, we want to perform this section step locally to create a mini_file which contains the cord_uid and the affliations information"


In [3]:
# # load the data columns and remove some reducdant information
# col_to_remove = ['text', 'bibliography']
# col_names = pd.read_csv('data/merged_raw_data.csv', nrows=1).columns.tolist()
# for c in col_to_remove:
#     col_names.remove(c)
# col_names = ['cord_uid', 'affiliations', 'location']

# # load the data
# df = pd.read_csv('data/merged_raw_data.csv', usecols=col_names)

# save
# df.to_csv('../data_location/merged_raw_data_location.csv')

# # check
# print(df.shape)
# df.head(5)

# Load the data and then identify their location

Methodology
- geograpy
    - geograpy1 or 2 did not work because of the errors in installation: https://github.com/Corollarium/geograpy2 has installation issues. DID NOT USE IT
    - geograpy3 works: https://github.com/jmbielec/geograpy3
- can GeoText as well: https://github.com/elyase/geotext
- look like the geograpy3 is more robust to bad text (although not 100% correct) that can detect more regions, but the precision may be low
- may be we should combine both the use of geograpy and geotext

Data
- GeoText country abbreivation to country full name
    - https://github.com/elyase/geotext/blob/master/geotext/data/countryInfo.txt
    - Need to remove the "#" from the header column. Created a countryInfo_revised.txt and save into the data folder

## Geotext 
- extract the Genotext abbreivation map

In [4]:
# # load Geotext location abbreivation
# df_geotext_loc = pd.read_csv('../data/data_location/countryInfo_revised.txt', 
#                                 comment='#', sep='\t')
# # create the dict
# geotext_loc_dict = dict(zip(df_geotext_loc['ISO'].tolist(),
#                            df_geotext_loc['Country'].tolist()
#                            ))

# # check
# print(geotext_loc_dict)
# df_geotext_loc.head(2)


## Classification
- Two version. One is no paralization, one is. If we run data_utils.extract_location(text, is_robust=True), highly recommend using the parallization version

In [19]:
from covid.data import utils as data_utils
from tqdm import tqdm_notebook as tqdm

In [20]:
# load data
df = pd.read_csv('../data/data_location/merged_raw_data_location.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df.dropna()
df = df.reset_index(drop=True)

# check
print(df.shape)
df.head(3)



(18558, 3)


,cord_uid,affiliations,location
0,gi6uaa83,Todd R Disotell (New York University ; 25 Wave...,25 Waverly Place ; 10003 ; New York ; NY ; USA
1,1wswi7us,Yee Leng Yap ; Xue Wu Zhang ; Antoine Danchin ...,"; ; 75724, Cedex 15 ; Paris ; France"
2,yy96yeu9,David Wang (University of California San Franc...,San Francisco ; California ; United States of ...


### non-parallelize

In [21]:
# cols = ['affiliations', 'location']

# # loop through each column's address and make classification
# for col in cols:
#     preds = []
#     for i in tqdm(range(0, df.shape[0])):
#         text = df.iloc[i][col]
#         # print(text)

#         # classification
#         countries = data_utils.extract_location(text, is_robust=True)
#         preds.append(countries)
        
#     # assign
#     df['%s_country'%col] = preds

# # save
# df.to_csv('../data/data_location/merged_raw_data_location_identified.csv')

### parallelize

In [17]:
import time

In [ ]:
cols = ['affiliations', 'location']
#num_cores = multiprocessing.cpu_count()
num_cores = 12

# loop through each column's address and make classification
for col in cols:
    print(col)
    preds = Parallel(n_jobs=num_cores)(delayed(data_utils.extract_location)(i) for i in df[col].tolist())

    # assign
    df['%s_country'%col] = preds

# save
df.to_csv('../data/data_location/merged_raw_data_location_identified.csv')


affiliations
